In [153]:
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
np.random.seed(123)
from scipy import io
import matplotlib.pyplot as plt
from skimage.transform import resize

In [154]:
dataset = io.loadmat('/content/Data_hoda_full.mat')

In [155]:
x_train_original = np.squeeze(dataset['Data'][0:1000])
y_train_original = np.squeeze(dataset['labels'][0:1000])
x_test_original = np.squeeze(dataset['Data'][1000:1200])
y_test_original = np.squeeze(dataset['labels'][1000:1200])

In [156]:
#resize
x_train_5by5 = [cv2.resize(img, (5, 5)) for img in x_train_original]
x_test_5by5 = [cv2.resize(img, (5, 5)) for img in x_test_original]

In [157]:
x_train = [x.reshape(25) for x in x_train_5by5]
x_test = [x.reshape(25) for x in x_test_5by5]

In [158]:
def print_data_info(x_train, y_train, x_test, y_test):
    #Check data Type
    print ("\ttype(x_train): {}".format(type(x_train)))
    print ("\ttype(y_train): {}".format(type(y_train)))

    #check data Shape
    print ("\tx_train.shape: {}".format(np.shape(x_train)))
    print ("\ty_train.shape: {}".format(np.shape(y_train)))
    print ("\tx_test.shape: {}".format(np.shape(x_test)))
    print ("\ty_test.shape: {}".format(np.shape(y_test)))

    #sample data
    print ("\ty_train[0]: {}".format(y_train[0]))

In [159]:
# Preprocess input data for Keras. 
x_train = np.array(x_train)
y_train = keras.utils.np_utils.to_categorical(y_train_original, num_classes=10)
x_test = np.array(x_test)
y_test = keras.utils.np_utils.to_categorical(y_test_original, num_classes=10)

In [160]:
print("Before Preprocessing:")
print_data_info(x_train_original, y_train_original, x_test_original, y_test_original)
print("After Preprocessing:")
print_data_info(x_train, y_train, x_test, y_test)

Before Preprocessing:
	type(x_train): <class 'numpy.ndarray'>
	type(y_train): <class 'numpy.ndarray'>
	x_train.shape: (1000,)
	y_train.shape: (1000,)
	x_test.shape: (200,)
	y_test.shape: (200,)
	y_train[0]: 6
After Preprocessing:
	type(x_train): <class 'numpy.ndarray'>
	type(y_train): <class 'numpy.ndarray'>
	x_train.shape: (1000, 25)
	y_train.shape: (1000, 10)
	x_test.shape: (200, 25)
	y_test.shape: (200, 10)
	y_train[0]: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [161]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [162]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=25))
model.add(Dense(10, activation='softmax'))

In [163]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 64)                1664      
                                                                 
 dense_22 (Dense)            (None, 10)                650       
                                                                 
Total params: 2,314
Trainable params: 2,314
Non-trainable params: 0
_________________________________________________________________


In [164]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [165]:
model.fit(x_train, y_train, epochs=30, batch_size=64, validation_split=0.2)

Epoch 1/30
13/13 [==============================] - 1s 34ms/step - loss: 2.2171 - accuracy: 0.1925 - val_loss: 2.0428 - val_accuracy: 0.3600
Epoch 2/30
13/13 [==============================] - 0s 6ms/step - loss: 2.0020 - accuracy: 0.4137 - val_loss: 1.8602 - val_accuracy: 0.5350
Epoch 3/30
13/13 [==============================] - 0s 7ms/step - loss: 1.8318 - accuracy: 0.5275 - val_loss: 1.6953 - val_accuracy: 0.6350
Epoch 4/30
13/13 [==============================] - 0s 4ms/step - loss: 1.6723 - accuracy: 0.6325 - val_loss: 1.5335 - val_accuracy: 0.6750
Epoch 5/30
13/13 [==============================] - 0s 4ms/step - loss: 1.5164 - accuracy: 0.6762 - val_loss: 1.3844 - val_accuracy: 0.7400
Epoch 6/30
13/13 [==============================] - 0s 4ms/step - loss: 1.3693 - accuracy: 0.7312 - val_loss: 1.2454 - val_accuracy: 0.7650
Epoch 7/30
13/13 [==============================] - 0s 5ms/step - loss: 1.2325 - accuracy: 0.7563 - val_loss: 1.1192 - val_accuracy: 0.7750
Epoch 8/30
13/13 [=

In [166]:
loss, acc = model.evaluate(x_test, y_test)
print('\nTesting loss: %.2f, acc: %.2f%%'%(loss, acc))

7/7 [==============================] - 0s 2ms/step - loss: 0.3484 - accuracy: 0.9150

Testing loss: 0.35, acc: 0.92%


In [168]:
# The predict_classes function outputs the highest probability class according to the trained classifier for each input example.
prModel = model.predict(x_test) 
classes_prediction = np.argmax(prModel,axis=1)
print("predicted:")
print(classes_prediction)
print("True Label:")
print(y_test_original)

predicted:
[7 2 3 1 5 5 4 7 3 3 0 8 8 0 2 9 3 6 7 4 0 3 6 3 9 2 7 5 2 9 2 5 5 8 9 2 5
 1 4 8 8 4 7 2 1 2 7 9 0 3 7 5 7 5 2 9 8 2 9 8 8 6 6 6 7 6 2 4 6 4 1 5 9 1
 8 4 0 5 6 2 4 3 2 7 7 7 7 1 8 1 7 8 7 7 8 9 7 2 3 1 0 2 9 6 3 5 5 0 0 9 6
 7 9 3 9 9 8 7 9 2 5 2 5 5 9 6 9 2 0 3 7 9 8 2 9 0 4 1 8 2 2 3 5 2 9 3 8 2
 7 0 9 9 0 7 6 2 4 7 9 3 7 0 7 1 9 4 7 3 4 1 5 6 7 9 1 3 5 4 5 7 4 1 3 3 1
 1 3 3 8 9 6 7 7 2 3 0 1 4 9 5]
True Label:
[7 2 3 1 5 5 4 7 3 2 0 8 8 0 2 9 3 6 7 4 0 3 6 3 9 2 7 5 2 9 7 5 5 8 9 6 5
 1 4 8 8 4 7 7 1 2 7 9 0 3 7 4 7 5 2 9 8 2 9 8 8 6 6 6 6 6 2 4 3 4 4 5 9 1
 8 2 0 5 6 2 4 3 2 7 7 7 7 1 8 1 7 8 7 7 8 9 3 2 3 1 0 2 9 6 3 5 5 0 0 3 6
 7 9 3 9 9 8 7 9 2 5 2 5 5 9 6 9 2 0 3 7 6 5 2 9 0 4 1 8 2 2 3 0 2 9 3 8 6
 7 0 9 9 0 7 6 5 4 7 9 3 7 0 7 1 9 4 7 3 4 1 5 6 7 9 1 3 5 4 5 7 4 1 3 3 1
 2 3 3 8 9 6 7 7 2 3 0 1 4 9 5]
